@property simplify the validity check of instance's attributes

## on-the-fly calculation 

In [110]:
from datetime import datetime, timedelta
import time

In [179]:
class Bucket(object):
    
    def __init__(self, period):
        self.period_delta = timedelta(seconds = period)
        self.reset_time = datetime.now()
        self.quota = 0
            
    def is_bucket_empty(self):
        now = datetime.now()
        if now - self.reset_time >= self.period_delta:
            print("Bucket is now empty!")
        else:
            time_left = self.period_delta - (now - self.reset_time)
            print("no, still full, but will be empty in %ss!" %time_left.seconds)
        
    def __repr__(self):
        return "Bucket(quota=%d)" % self.quota

In [180]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        print("the bucket is empty")
        bucket.quota = 0
        bucket.reset_time = now
    
    print("refilling the bucket")
    bucket.quota += amount

In [181]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota - amount < 0:
        return False
    bucket.quota -= amount
    return True

##### how to use the class 

In [193]:
time_to_empty_bucket = 10 #s

In [194]:
bucket = Bucket(time_to_empty_bucket)
print("Time it takes to the bucket to empty itself: ", bucket.period_delta)
print("Last time the bucket was filled: ", bucket.reset_time)

Time it takes to the bucket to empty itself:  0:00:10
Last time the bucket was filled:  2016-01-08 13:25:30.708535


In [195]:
bucket.is_bucket_empty()

no, still full, but will be empty in 9s


In [196]:
print(bucket)

Bucket(quota=0)


If I take more than *time_to_empty_bucket* to fill the bucket, the bucket will be empty

In [197]:
fill(bucket, 50)

refilling the bucket


Then I deduct the quota I need to fully fill it (%)

In [198]:
quota_requested = 45
if deduct(bucket, quota_requested):
    print('Had %d quota' %quota_requested)
else:
    print('Not enough for %d quota' %quota_requested)
print(bucket)

Had 45 quota
Bucket(quota=5)


The problem with this implementation is that **deduct** return false for 2 raisons. 
 * the bucket is now empty
 * the quota requested is too hight

We need to keep track of ht max_quota and quota_consumed

In [199]:
class Bucket(object):
    
    def __init__(self, period):
        self.period_delta = timedelta(seconds = period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return ('Bucket(max_quota=%d, quota_consumed=%d)' %(self.max_quota, self.quota_consumed))
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            #quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            #quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [200]:
bucket = Bucket(100)
print('Initial', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)


In [201]:
fill(bucket, 100)
print('Filled', bucket)

refilling the bucket
Filled Bucket(max_quota=100, quota_consumed=0)


In [202]:
quota_requested = 45
if deduct(bucket, quota_requested):
    print('Had %d quota' %quota_requested)
else:
    print('Not enough for %d quota' %quota_requested)
print(bucket)


Had 45 quota
Bucket(max_quota=100, quota_consumed=45)


In [203]:
print('Now ', bucket)

Now  Bucket(max_quota=100, quota_consumed=45)


In [205]:
quota_requested = 55
if deduct(bucket, quota_requested):
    print('Had %d quota' %quota_requested)
else:
    print('Not enough for %d quota' %quota_requested)
print(bucket)


Not enough for 55 quota
Bucket(max_quota=100, quota_consumed=45)


In [206]:
print('Still', bucket)

Still Bucket(max_quota=100, quota_consumed=45)
